In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

import requests
import json
import os
from datetime import datetime

# --- CONFIGURATION ---
# The URL for real-time crypto market data (USD prices, market cap, volume)
API_URL = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false"

# --- HELPER FUNCTIONS ---
def get_api_data(url):
    """
    Connects to the API and returns the raw JSON data.
    """
    try:
        response = requests.get(url)
        response.raise_for_status() # Check if the request was successful
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def save_to_lakehouse(data, folder_name="crypto_raw"):
    """
    Saves the JSON data to the Lakehouse 'Files' section (Bronze Layer).
    """
    # 1. Create a filename with the current timestamp
    # This ensures we don't overwrite old data (History preservation)
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"crypto_data_{current_time}.json"
    
    # 2. Define the path. 
    # "/lakehouse/default/Files/" is the standard path to your attached Lakehouse
    folder_path = f"/lakehouse/default/Files/bronze/{folder_name}"
    full_path = f"{folder_path}/{file_name}"

    # 3. Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)

    # 4. Write the file
    with open(full_path, "w") as f:
        json.dump(data, f)
    
    print(f"SUCCESS: Data saved to {full_path}")

# --- MAIN EXECUTION ---
print("Starting ingestion...")
data = get_api_data(API_URL)

if data:
    save_to_lakehouse(data)
    print("Ingestion complete.")
else:
    print("Ingestion failed.")

StatementMeta(, 5f37d996-7631-4410-ab01-0075910fe426, 3, Finished, Available, Finished)

Starting ingestion...
SUCCESS: Data saved to /lakehouse/default/Files/bronze/crypto_raw/crypto_data_20260119_054030.json
Ingestion complete.


In [3]:
import requests
import json
import os
from datetime import datetime
from notebookutils import mssparkutils

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
# PASTE YOUR PATH INSIDE THESE QUOTES 👇
LAKEHOUSE_ROOT_PATH = "abfss://DE_Project_Portfolio@onelake.dfs.fabric.microsoft.com/Lh_Medallion.Lakehouse/Files"

API_URL = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false"

# ==============================================================================
# 2. THE INGESTION LOGIC
# ==============================================================================
def get_api_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"API Error: {e}")
        return None

def save_to_onelake(data, root_path):
    # Create filename with timestamp
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"crypto_data_{current_time}.json"
    
    # Construct the FULL OneLake URL
    # We manually build the path: Root -> bronze folder -> crypto_raw folder -> file
    full_path = f"{root_path}/bronze/crypto_raw/{file_name}"

    print(f"Attempting to write to: {full_path}")

    # Write using Microsoft Spark Utilities (The "Cloud Native" way)
    try:
        mssparkutils.fs.put(full_path, json.dumps(data), True)
        print("✅ SUCCESS! File written to OneLake.")
        return full_path
    except Exception as e:
        print("❌ WRITE ERROR: Could not save to OneLake.")
        print(e)
        return None

# ==============================================================================
# 3. EXECUTION & VERIFICATION
# ==============================================================================
print("Starting Ingestion...")

data = get_api_data(API_URL)

if data:
    saved_path = save_to_onelake(data, LAKEHOUSE_ROOT_PATH)

    # Double check: List the file immediately to prove it exists
    if saved_path:
        print("\n--- 👀 VERIFYING FILE EXISTS IN STORAGE ---")
        try:
            folder = f"{LAKEHOUSE_ROOT_PATH}/bronze/crypto_raw"
            files = mssparkutils.fs.ls(folder)
            for f in files:
                print(f"Confirmed file found: {f.name}")
        except Exception as e:
            print(f"Verification failed: {e}")

StatementMeta(, 5f37d996-7631-4410-ab01-0075910fe426, 5, Finished, Available, Finished)

Starting Ingestion...
Attempting to write to: abfss://DE_Project_Portfolio@onelake.dfs.fabric.microsoft.com/Lh_Medallion.Lakehouse/Files/bronze/crypto_raw/crypto_data_20260119_055032.json
✅ SUCCESS! File written to OneLake.

--- 👀 VERIFYING FILE EXISTS IN STORAGE ---
Confirmed file found: crypto_data_20260119_055032.json
